<a href="https://colab.research.google.com/github/willermo/video-enhancer/blob/main/video-enhancer-colab-notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video enhancer pipeline

Automated AI Video Enhancement and Upscaling Pipeline.

This script provides a complete, end-to-end solution for restoring low-quality
videos. It is designed to fix common issues like blurriness, compression
artifacts, and low-resolution faces by using a 2-stage AI process:

1.  **Real-ESRGAN:** Upscales the video resolution (default 4x), enhancing
    general texture and detail.
2.  **GFPGAN:** Performs a second pass to specifically identify and restore
    human faces, fixing "waxy" or blurry features into a realistic result.

## Python 3.10 interpreter installation

To satisfy the dependencies we must use Python 3.10

In future we will try to compose a more up-to-date dependencies hierarchy, but for now, we are good as we are :)

In [1]:
# Install Python 3.10
!echo "--- Checking GPU status ---"
!nvidia-smi

!echo "--- Installing Python 3.10 and venv ---"
# We add "quiet" flags (-qq) to keep the output clean
!sudo apt-get update -qq
!sudo apt-get install -y python3.10 python3.10-venv -qq

!echo "--- Setup complete ---"

--- Checking GPU status ---
Sat Nov  1 01:52:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------

## Cloning GitHub Repository

This will pull the entire project (enhance.py, requirements.txt, etc.) into the Colab environment.

In [4]:
# Clone Your Repository
!echo "--- Cloning project from GitHub ---"
# !!!
# !!! REPLACE THIS URL WITH YOUR OWN GITHUB REPO URL
# !!!
!git clone https://github.com/willermo/video-enhancer.git

# Move into the new project directory
%cd video-enhancer

!echo "--- Project cloned successfully ---"

--- Cloning project from GitHub ---
Cloning into 'video-enhancer'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 15 (delta 4), reused 15 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 11.98 KiB | 11.98 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/video-enhancer
--- Project cloned successfully ---


## Install Dependencies
This cell creates the Python 3.10 virtual environment and installs all the correct dependencies from the requirements.txt file.

In [5]:
# Create venv and install requirements
!echo "--- Creating Python 3.10 venv ---"
!python3.10 -m venv venv

!echo "--- Activating venv and installing dependencies (this may take 2-3 minutes) ---"
# This activates the venv AND runs pip install in one line
!source venv/bin/activate && pip install -r requirements.txt

!echo "--- Environment is ready ---"

--- Creating Python 3.10 venv ---
--- Activating venv and installing dependencies (this may take 2-3 minutes) ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 117.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 KB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.4/292.4 KB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.3/770.3 KB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## ⚠️ MANUAL - Upload Your Video Chunk
This is a manual step.

In the file sidebar on the left, navigate inside the `video-enhancer` directory.

Drag and drop your first video chunk (e.g., `chunk_01.mkv`) into that directory.

Wait for the upload to complete.

(You can use the `scripts/split_video.sh` on your local machine to create these chunks)

## Run the Enhancement (The GPU Part)
This is the main event. This command activates the `venv` and runs the `enhance.py` script. It will automatically detect the NVIDIA GPU.

Note: We do not use the `--parallel` flag, as we want the script to run in its default, single-threaded GPU mode.

In [ ]:
# Run the script

# ---
# !!!
# !!! RENAME INPUT_FILE and OUTPUT file appropriately
# !!!
%env INPUT_FILE=2.Lorenzoni_chunk_00.mkv
%env OUTPUT_FILE=2.Lorenzoni_chunk_00_restored.mkv
# ---

!echo "--- Starting AI Enhancement Pipeline ---"
!echo "--- This will be fast, but not instant. (Approx. 1 hour for a 5-min chunk) ---"

!source venv/bin/activate && python enhance.py \
    --input $INPUT_FILE \
    --output $OUTPUT_FILE

!echo "--- Processing complete! Your file is ready. ---"

env: INPUT_FILE=2.Lorenzoni_chunk_00.mkv
env: OUTPUT_FILE=2.Lorenzoni_chunk_00_restored.mkv
--- Starting AI Enhancement Pipeline ---
--- This will be fast, but not instant. (Approx. 1 hour for a 5-min chunk) ---
Initializing AI models...
NVIDIA GPU detected: Tesla T4. Running in single-threaded GPU mode.
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/video-enhancer/venv/lib/python3.10/site-packages/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:00<00:00, 156MB/s]
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/video-enhancer/gfpgan/weights/detection_Resnet50_Final.pth

100% 104M/104M [00:00<00:00, 327MB/s] 
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" to /content/video-enhancer/gfpgan/weights/parsing_parsenet.pth

100% 81.4M/81.4M [00:00<00:00, 305MB/s]
Downloading: "https://github.com/TencentARC/

## ⚠️ MANUAL - Download Your Restored File

In the file sidebar, click the "Refresh" button.

You will see your new file (e.g., `restored_chunk_01.mp4`) in the `video-enhancer` directory.

Click the three dots next to the file and select **"Download"**.

## ⚠️ MANUAL - Clean Up & Repeat
After you've downloaded your file, run this cell to delete the large video files from Colab's temporary storage, freeing up space for your next chunk.

## Looping

Repeat the flow:
- **upload**
- **enhance**
- **download**
